In [1]:
import os
import sys
import socket
import re
import numpy as np
import string
import warnings
from timeit import default_timer as timer
from datetime import datetime
from glob import glob

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,desc,row_number,col,year,month,dayofmonth,dayofweek,to_timestamp,size,isnan,lit,lower
import pyspark.sql.functions as F
from pyspark.sql.types import MapType, StringType, IntegerType, StructType, StructField, FloatType, ArrayType

# Config

In [2]:
country_code = "US"
language_code = "en"
print('Country:', country_code)
print('Language:', language_code)

wordlists=['loss', 'unemployed', 'search', 'hire', 'offer']

Country: US
Language: en


In [3]:
try:
    spark
except NameError:
    if 'samuel' in socket.gethostname().lower():
        print('Create Local SparkSession')
        spark = SparkSession.builder.config(
        "spark.driver.host", "localhost").appName(
        "get-tweets-mentions").getOrCreate()
    else:
        print('Create Cluster SparkSession')
        spark = SparkSession.builder.appName(
        "get-tweets-mentions").getOrCreate()
    
# Local
print('Hostname:', socket.gethostname())
if  'samuel' in socket.gethostname().lower():
    path_to_data='../../data'
# Cluster
else:
    path_to_data='/user/spf248/twitter/data'
    
path_to_timelines=os.path.join(path_to_data,'timelines','extract',country_code)
path_to_mentions=os.path.join(path_to_data,'mentions')
path_to_keywords=os.path.join(path_to_data,'keywords','labor',country_code)
print(path_to_timelines)
print(path_to_mentions)
print(path_to_keywords)

Hostname: Samuels-MacBook-Pro.local
../../data/timelines/extract/US
../../data/mentions
../../data/keywords/labor/US


# Data Processing

In [4]:
print('Import Datasets')
df=spark.read.parquet(path_to_timelines)

print("REPARTITION")
df=df.repartition(1000)

df.cache()

Import Datasets
REPARTITION


DataFrame[tweet_id: string, created_at: timestamp, text: string, tweet_lang: string, user_id: string, user_location: string, place_id: string, tweet_longitude: double, tweet_latitude: double]

In [121]:
print("SELECT LANGUAGE")
df=df.where(df.tweet_lang==language_code)

print("EXTRACT YEAR AND MONTH")
df=df.withColumn('year',year('created_at').cast("string"))
df=df.withColumn('month',month('created_at').cast("string"))

for wordlist in wordlists:
    
    print(wordlist)
    targets=list(spark.read.csv(os.path.join(path_to_keywords,country_code+'-'+wordlist+'.txt')).toPandas()['_c0'])
    print('# targets:',len(targets))
#     print('\n'.join(targets))
    
    # Create an indicator equal to one if a tweet contains any of the target sentence and zero otherwise
    df=df.withColumn(wordlist,(sum([df.text.contains(target).cast("int") for target in targets])>0).cast("int"))
    
df=df.drop(*['tweet_id','tweet_lang','created_at','text','place_id','tweet_longitude','tweet_latitude'])

print("COUNT MENTIONS BY YEAR, MONTH, LOCATION, AND USER")
df=df.groupBy('year','month','user_location','user_id').sum()

for wordlist in wordlists:
    
    # Binarize mention multiplicity at the user level
    df=df.withColumn(wordlist,(df['sum('+wordlist+')']>0).cast("int"))
    df=df.drop('sum('+wordlist+')')
    
print("COUNT USERS BY YEAR, MONTH, LOCATION")
col2func=dict((wordlist,'sum') for wordlist in wordlists)
col2func.update({'user_id':'count'})
df=df.groupBy('year','month','user_location').agg(col2func)

print("RENAME")
df=df.select([col(name).alias(name.replace('count(','').replace('sum(','').replace(')','')) for name in df.columns])
df=df.withColumnRenamed('user_id','n_users')
for wordlist in wordlists:
    df=df.withColumnRenamed(wordlist,'n_'+wordlist)

SELECT LANGUAGE
EXTRACT YEAR AND MONTH
loss
# targets: 36
unemployed
# targets: 16
search
# targets: 50
hire
# targets: 29
offer
# targets: 9
COUNT MENTIONS BY YEAR, MONTH, LOCATION, AND USER
COUNT USERS BY YEAR, MONTH, LOCATION
RENAME


In [123]:
print('SAVE')
start = timer()

df.write.mode("overwrite").parquet(os.path.join(path_to_mentions,country_code))

end = timer()
print('DONE IN', round(end - start), 'SEC')

SAVE
DONE IN 163 SEC


In [1]:
print('Computing Time (in hour):',round((1580214760905-1580213691649)/(1000*3600),2))

Computing Time (in hour): 0.3
